In [1]:
import numpy as np
import regex as re
import os
import math
import pandas as pd 
import sys
from openpyxl import load_workbook, Workbook
from openpyxl.utils.cell import get_column_letter

path_src = "/local/scratch/Figaro/figaro-code"
exp_names = ["mkl", "figaro"]
exp_paths = {"figaro": "dumps/figaro", "mkl": "dumps/python/numpy"}
csv_name = "R.csv"

row_nums = [512]
column_nums =[2]
row_nums = [512, 1024, 2048, 4096]
column_nums = [1, 2, 4, 8, 16, 32, 64, 128, 256]
row_col_nums = []
np.set_printoptions(threshold=sys.maxsize, precision=14)
join_order = "FullJoin"

for row_num in row_nums:
    for col_num in column_nums:
        row_col_nums.append((row_num, col_num))

def arrange_up_triang(n: int):
    A  = np.zeros( (n, n))
    idx = 1
    for row_idx in range(n):
        for col_idx in range(row_idx, n):
            A[row_idx, col_idx] = idx
            idx += 1
    return A


def compare_rs(R: np.array, num_rows: int, num_cols: int):
    R_expX = arrange_up_triang(num_cols) * math.sqrt(num_rows)
    diff = R[:num_cols, :num_cols] - R_expX
    #print(R[:num_cols, :num_cols])
    #print(R_expX)
    #print(diff)
    rel_fro = np.linalg.norm(diff, ord='fro') / np.linalg.norm(R_expX, ord='fro')
    return rel_fro

exp_rel_errors = {}

for exp_name in exp_names:
    #Initialize paths for the current experiment 
    exp_path = exp_paths[exp_name]
    accur_path = os.path.join(path_src, exp_path)
    df_measurement = pd.DataFrame(columns=column_nums)
    dump_r_path = os.path.join(accur_path, csv_name)


    for db_idx, db_num in enumerate(range(1, len(row_nums) * len(column_nums) + 1)): 
        db_name = "DBCartesianProductAccuracy{}".format(db_num)
        path_csv = os.path.join(accur_path, db_name, join_order, csv_name)
        
        # Writng header files
        row_num = row_col_nums[db_idx][0]
        col_num = row_col_nums[db_idx][1]
        r_mat = np.genfromtxt(path_csv, delimiter=',')
        rel_error = compare_rs(r_mat, row_num, col_num)
        df_measurement.at[row_num, col_num] = rel_error
    
    exp_rel_errors[exp_name] = df_measurement
    df_measurement.to_csv(exp_name+"comp.csv")
print(exp_rel_errors)

{'mkl':               1            2            4            8            16   \
512   2.98318e-15  1.51644e-14  2.66357e-14  2.32415e-14  3.59507e-14   
1024  3.35287e-14  3.77011e-14   4.6234e-14  1.06755e-13   1.1012e-13   
2048  2.08822e-14  8.77204e-14  1.01512e-13  2.69814e-13  2.88897e-13   
4096  8.68194e-14  1.71403e-13  6.69545e-13  4.82761e-13  8.23772e-13   

              32           64           128          256  
512   2.86957e-14  2.37243e-14  2.19853e-14  3.10458e-14  
1024  8.35344e-14  6.93354e-14  6.18664e-14  6.06355e-14  
2048  2.69385e-13  1.93979e-13  1.73573e-13  1.48883e-13  
4096  6.41157e-13  5.52684e-13  4.62074e-13  4.28584e-13  , 'figaro':               1            2            4            8            16   \
512   9.42055e-16  7.91747e-16  1.52709e-15  1.72339e-15  1.99372e-15   
1024            0  8.30815e-16   1.5843e-16   1.2393e-15  1.93254e-15   
2048  1.25607e-15  8.80213e-16  2.57364e-15  4.84873e-15  3.06877e-15   
4096  1.55431e-15  4.15407e-